In [41]:
#!pip install jupysql pandas matplotlib duckdb-engine duckdb

In [42]:
import duckdb
import pandas as pd

%load_ext sql
%sql duckdb:///:memory:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [43]:
%%sql
CREATE TABLE Riders             AS FROM "../data/riders.csv";
CREATE TABLE RiderTeams         AS FROM "../data/rider_teams.csv";
CREATE TABLE Managers           AS FROM "../data/managers.csv";
CREATE TABLE ManagerTeams       AS FROM "../data/manager_teams.csv";
CREATE TABLE ManagerTeamsCheapo AS FROM "../data/manager_cheapo_teams.csv";
CREATE TABLE PointsSystem       AS FROM "../data/points_system.csv";
CREATE TABLE Races              AS FROM "../data/races.csv";
CREATE TABLE Results_2023       AS FROM "../data/results/results_2023_full.csv";
CREATE TABLE Results_2024       AS FROM "../data/results/results_2024_full.csv";
CREATE TABLE Results_2025       AS FROM "../data/results/results_2025_full.csv";
CREATE TABLE CheapoBans         AS FROM "../data/cheapo_bans.csv";

Running query in 'duckdb:///:memory:'

RuntimeError: (duckdb.duckdb.CatalogException) Catalog Error: Table with name "Riders" already exists!
[SQL: CREATE TABLE Riders             AS FROM "../data/riders.csv";]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [53]:
%%sql
CREATE OR REPLACE TABLE RaceResultsPoints AS
SELECT
     results.Race
    ,races.RaceName
    ,races.RaceCategory
    ,results.Year
    ,results.Stage_ID
    ,results.Stage_Date
    ,results.Rider AS RiderName_PCS
    ,riders.RiderName_Zweeler
    ,results.Rank
    ,points.RacePoints AS Points
    ,CASE WHEN races.RaceName = 'Tour Down Under' THEN NULL Else points.RacePoints END AS CheapoPoints
FROM Results_2025 results
INNER JOIN Races races ON races.RaceName_PCS = results.Race
INNER JOIN Riders riders ON riders.RiderName_PCS = results.Rider
INNER JOIN PointsSystem points ON 1 = 1
    AND points.RaceRank = TRY_CAST(results.Rank AS INTEGER) 
    AND CASE WHEN results.Stage_ID LIKE '%stage%' THEN REPLACE(races.RaceCategory, 'Tour', 'Stage') ELSE races.RaceCategory END = points.RaceCategory
WHERE 1 = 1
    AND results.Stage_Type <> 'TTT';

SELECT *
FROM RaceResultsPoints
WHERE RiderName_PCS LIKE '%Welsford%';

Running query in 'duckdb:///:memory:'

RACE,RaceName,RaceCategory,YEAR,STAGE_ID,STAGE_DATE,RiderName_PCS,RiderName_Zweeler,RANK,Points,CheapoPoints
tour-down-under,Tour Down Under,Tour Cat. 2,2025,stage-1,2025-01-21,Welsford Sam,"Welsford, Sam",1,25.0,None
tour-down-under,Tour Down Under,Tour Cat. 2,2025,stage-2,2025-01-22,Welsford Sam,"Welsford, Sam",1,25.0,None
tour-down-under,Tour Down Under,Tour Cat. 2,2025,stage-6,2025-01-26,Welsford Sam,"Welsford, Sam",1,25.0,None
uae-tour,UAE Tour,Tour Cat. 2,2025,stage-5,2025-02-21,Welsford Sam,"Welsford, Sam",4,14.0,14.0
uae-tour,UAE Tour,Tour Cat. 2,2025,stage-6,2025-02-22,Welsford Sam,"Welsford, Sam",9,4.0,4.0


In [55]:
%%sql
SELECT *
FROM RaceResultsPoints
WHERE Rider LIKE '%Onley%'

Running query in 'duckdb:///:memory:'

RuntimeError: If using snippets, you may pass the --with argument explicitly.
For more details please refer: https://jupysql.ploomber.io/en/latest/compose.html#with-argument


Original error message from DB driver:
(duckdb.duckdb.BinderException) Binder Error: Referenced column "Rider" not found in FROM clause!
Candidate bindings: "RiderName_PCS", "RiderName_Zweeler", "RaceName", "RaceCategory"

LINE 3: WHERE Rider LIKE '%Onley%'
              ^
[SQL: SELECT *
FROM RaceResultsPoints
WHERE Rider LIKE '%Onley%']
(Background on this error at: https://sqlalche.me/e/20/f405)

If you need help solving this issue, send us a message: https://ploomber.io/community
